##Scraping code for Google Colab. Web extraction from '*bolsasymercados.es*'

#Instalation of Selenium and chromium in the instance

In [ ]:
!pip install selenium

In [ ]:
!apt-get update

In [ ]:
!apt-get install -y chromium-browser

In [ ]:
!apt install chromium-chromedriver

In [5]:
!ls /usr/lib/chromium-browser #Finding the path of the driver

chromedriver


In [6]:
import requests
import time
import csv
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

#Definition of the options for the driver.

In [7]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

#Function for obtaining every fix information in a page.

In [8]:
def pagesource(url, driver):
    driver = driver
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

In [9]:
driver = driversetup()
url = 'https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen'
soup= pagesource (url, driver)
#print(soup.prettify())

#Using Xpath for dynamic information

In [ ]:
driver = driversetup()
url = 'https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen'
driver.get(url)
time.sleep(5)
table= driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr")
#print(table)

for row in table:
  print(row.text)


In [12]:
driver = driversetup()
url = 'https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen'
driver.get(url)
time.sleep(5) # with this line we delay the access to the web avoiding permisions' problems

index=[ind.text for ind in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[1]") ]
lasts=[last.text for last in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[2]") ]
difs=[dif.text for dif in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[3]") ]
maxs=[max.text for max in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[4]") ]
mins=[min.text for min in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[5]") ]
dates=[date.text for date in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[6]") ]
times=[time.text for time in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[7]") ]
difs_year=[dif_year.text for dif_year in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[8]") ]

In [13]:
#We create a DataFrame to save the data.
df=pd.DataFrame(columns=["Index","Last","Dif","Maxim","Minim","Date","Hour","Dif_year"])

df["Index"]=index
df["Last"]=lasts
df["Dif"]=difs
df["Maxim"]=maxs
df["Minim"]=mins
df["Date"]=dates
df["Hour"]=times
df["Dif_year"]=difs_year

df.head()

,Index,Last,Dif,Maxim,Minim,Date,Hour,Dif_year
0,IBEX 35®,"9.615,40","0,70%","9.642,00","9.590,80",15/09/2023,11:49:49,"16,85%"
1,IBEX 35® con Dividendos,"31.768,80","0,69%","31.856,70","31.687,80",15/09/2023,11:49:49,"20,64%"
2,IBEX MEDIUM CAP®,"13.494,20","0,33%","13.535,20","13.489,30",15/09/2023,11:48:55,"5,44%"
3,IBEX SMALL CAP®,"7.905,30","0,02%","7.949,30","7.901,70",15/09/2023,11:48:49,"10,02%"
4,IBEX 35® Bancos,"624,50","0,66%","625,70","621,40",15/09/2023,11:49:49,"19,29%"


In [15]:
#Saving the table as csv
df.to_csv("Ejemplo_scraping1.csv", encoding='utf-8')